<a href="https://colab.research.google.com/github/juanllm-code/popt/blob/main/popt_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

In this colab, we benchmark [POPT](https://medium.com/@juan.sunnyvale/popt-why-preference-optimization-works-and-why-it-wont-take-over-the-world-d550b1cacad7) and [DPO](https://arxiv.org/abs/2305.18290) for fine-tuning LLMs. To run this colab, you will need to connect to a high-RAM A100 GPU.

In this colab, due to compute constraints, we fine-tune with 500 samples taken from the trl-lib/ultrafeedback_binarized dataset. The dataset lends itself to work with DPO, but with some minor data-prep, we make it work for POPT.

# Step 1: Install Dependencies and Verify you have GPUs

The first cell may take a couple of mins to run. You may be prompted to restart the colab, and if so, please restart. One of the libraries contains the functions that we will use to score DPO and POPT predictions, and the other library (trl) contains the implementation of DPO. It is pinned to a specific version because trl changes very rapidly, and hence this code may not work for past versions.

The second cell runs nvidia-smi to read available GPUs. If you do not have at least one A100 available, this code may not run.

In [ ]:
!pip install trl==0.15.2 rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 14 03:11:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             37W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# POPT Trainer Implementation

Here we define a simple POPT trainer. The most important part and contribution here is the loss function, which follows the formula proposed and proven [here](https://medium.com/@juan.sunnyvale/popt-the-preference-optimized-policy-theorem-b0ab20970518)

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import torch
import os
import logging
import json
import gc
import time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import DPOConfig, DPOTrainer
from datetime import datetime
from rouge_score import rouge_scorer

import torch
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
import os
import json
import time

class POPTTrainer(Trainer):
    """Preference-Optimized Policy Trainer (POPT) implementation"""

    def __init__(self, model, ref_model, beta=1.0, *args, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.ref_model = ref_model
        self.beta = beta
        self.metrics_history = {'loss': [], 'kl_div': [], 'reward': []}

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
      """Custom loss function for POPT"""
      # Move inputs to device if needed
      inputs = {k: v.to(model.device) if hasattr(v, 'to') else v for k, v in inputs.items()}

      # Get model outputs
      outputs = model(**inputs)
      logits = outputs.logits

      # Get reference model outputs
      with torch.no_grad():
          ref_outputs = self.ref_model(**inputs)
          ref_logits = ref_outputs.logits

      # Compute KL divergence (normalized)
      log_probs = F.log_softmax(logits, dim=-1)
      ref_log_probs = F.log_softmax(ref_logits, dim=-1)

      kl_div = ((log_probs - ref_log_probs).sum(dim=-1).mean()) / log_probs.shape[-1]

      # Compute reward (use mean instead of sum)
      reward = log_probs.mean()

      # Final loss (increase beta)
      loss = -reward + self.beta * kl_div
      print(f"KL={kl_div}, Reward={reward}, Loss={loss}")

      # Store metrics for evaluation
      if not getattr(self, "is_in_train", True):
          with torch.no_grad():
              self.metrics_history['loss'].append(float(loss.item()))
              self.metrics_history['kl_div'].append(float(kl_div.item()))
              self.metrics_history['reward'].append(float(reward.item()))

      return (loss, outputs) if return_outputs else loss

# Data loading

Nothing fancy. Just downloading the data. It will take a couple of mins

In [ ]:
# ===== DATA LOADING =====
def prepare_datasets(tokenizer):
    """Load and prepare dataset"""
    logger.info(f"Loading dataset: {CONFIG['dataset_name']}")
    dataset = load_dataset(CONFIG["dataset_name"], split=CONFIG["dataset_split"])

    # ✅ Limit training set to 500 samples
    train_data = dataset.select(range(CONFIG["num_train_samples"]))

    # ✅ Select separate evaluation samples
    eval_data = dataset.select(range(CONFIG["num_train_samples"], CONFIG["num_train_samples"] + CONFIG["eval_samples"]))

    logger.info(f"Training samples: {len(train_data)}")
    logger.info(f"Evaluation samples: {len(eval_data)}")

    return train_data, eval_data

# Training functions

Here we define the functions to fine tune using DPO and POPT. Some things for people to play around with if hardware allows are:

- Batch size: Keeping it pretty low per GPU (1 or 2) to avoid OOMs
- Gradient accumulation (the higher the batch size, the less you need to think about this).
- max_prompt_length: We could go for the full prompts, although we are restricting it here

For DPO, the data from the dataset is essentially "plug-and-play", so most of what we do is set up configs here. However, the train_popt function is a bit more involved because it requires putting the data in a pytorch dataset.

In [ ]:
# ===== TRAINING FUNCTIONS =====
def train_dpo(tokenizer, train_data, eval_data):
    """Train a model using DPO and evaluate after training."""
    logger.info("Starting DPO training")

    try:
        # ✅ Free GPU memory before training
        torch.cuda.empty_cache()
        gc.collect()

        # ✅ Load model with gradient checkpointing
        model = AutoModelForCausalLM.from_pretrained(CONFIG["model_name"]).to(device)
        model.gradient_checkpointing_enable()  # ✅ Enable memory-efficient training

        # ✅ Reduce batch size to 1 for lower memory usage
        dpo_config = DPOConfig(
            output_dir=os.path.join(CONFIG["output_dir"], "dpo"),
            per_device_train_batch_size=2,  # ✅ Small batch size
            gradient_accumulation_steps=4,  # ✅ Accumulate gradients to compensate
            learning_rate=CONFIG["learning_rate"],
            num_train_epochs=CONFIG["num_train_epochs"],
            beta=CONFIG["beta"],
            logging_steps=10,
            save_strategy="no",
            evaluation_strategy="no",
            max_prompt_length=CONFIG["max_length"] // 2,
            padding_value=tokenizer.pad_token_id,
            fp16=True,  # ✅ Mixed precision for H100
        )

        # ✅ Train with DPOTrainer
        dpo_trainer = DPOTrainer(
            model=model,
            args=dpo_config,
            train_dataset=train_data,
            tokenizer=tokenizer,
        )

        # Train model
        start_time = time.time()
        dpo_trainer.train()
        training_time = time.time() - start_time
        logger.info(f"DPO training completed in {training_time:.2f} seconds")

        # ✅ Evaluate model after training
        metrics, samples = evaluate_model(model, tokenizer, eval_data, "DPO")
        metrics["training_time"] = training_time

        # ✅ Save results
        results = {
            "method": "DPO",
            "evaluation_metrics": metrics,
            "samples": samples,
            "training_time": training_time,
            "model_size": sum(p.numel() for p in model.parameters()),
            "parameters": {
                "beta": CONFIG["beta"],
                "learning_rate": CONFIG["learning_rate"],
                "batch_size": 1,  # ✅ Use batch_size=1
                "epochs": CONFIG["num_train_epochs"]
            }
        }

        with open(os.path.join(CONFIG["output_dir"], "dpo_results.json"), "w") as f:
            json.dump(results, f, indent=2)

        return model, metrics, results

    except Exception as e:
        logger.error(f"Error in DPO training: {e}")
        logger.exception("Details:")
        results = {
            "method": "DPO",
            "error": str(e),
            "training_time": 0.0,
            "evaluation_metrics": {
                "rouge1": 0.0,
                "rouge2": 0.0,
                "rougeL": 0.0
            }
        }
        return None, None, results

def train_popt(tokenizer, train_data, eval_data):
    """Train a model using POPT method"""
    logger.info("Starting POPT training")

    try:
        # ✅ Free GPU memory before training
        torch.cuda.empty_cache()
        gc.collect()

        # ✅ Load models
        model = AutoModelForCausalLM.from_pretrained(CONFIG["model_name"]).to(device)
        ref_model = AutoModelForCausalLM.from_pretrained(CONFIG["model_name"]).to(device)

        # ✅ Enable memory optimization
        model.gradient_checkpointing_enable()

        # ✅ Convert `train_data` into a dataset (since we removed `train_dataset`)
        class PreferenceDataset(torch.utils.data.Dataset):
            def __init__(self, data, tokenizer):
                self.data = data
                self.tokenizer = tokenizer

            def __len__(self):
                return len(self.data)

            def __getitem__(self, idx):
                item = self.data[idx]
                user_input = next(msg["content"] for msg in item["chosen"] if msg["role"] == "user")
                chosen_output = next(msg["content"] for msg in item["chosen"] if msg["role"] == "assistant")

                tokens = self.tokenizer(
                    user_input,
                    max_length=CONFIG["max_length"],
                    padding="max_length",
                    truncation=True,
                    return_tensors="pt",
                )

                return {
                    "input_ids": tokens["input_ids"][0],
                    "attention_mask": tokens["attention_mask"][0],
                    "labels": self.tokenizer(
                        chosen_output,
                        max_length=CONFIG["max_length"],
                        padding="max_length",
                        truncation=True,
                        return_tensors="pt",
                    )["input_ids"][0],
                }

        # ✅ Convert train_data into a PyTorch dataset
        train_dataset = PreferenceDataset(train_data, tokenizer)

        # ✅ Set up training arguments
        training_args = TrainingArguments(
            output_dir=os.path.join(CONFIG["output_dir"], "popt"),
            per_device_train_batch_size=2,  # ✅ Small batch size to prevent OOM
            gradient_accumulation_steps=4,  # ✅ Stabilizes training
            learning_rate=CONFIG["learning_rate"],
            num_train_epochs=CONFIG["num_train_epochs"],
            logging_dir=os.path.join(CONFIG["output_dir"], "popt_logs"),
            logging_steps=10,
            save_strategy="epoch",
            save_total_limit=1,
            evaluation_strategy="no",
            fp16=True,  # ✅ Mixed precision for memory efficiency
            max_grad_norm=1.0,
            weight_decay=0.01,
            remove_unused_columns=False,
        )

        # ✅ Create POPT trainer
        trainer = POPTTrainer(
            model=model,
            ref_model=ref_model,
            beta=CONFIG["beta"],
            args=training_args,
            train_dataset=train_dataset,
        )

        # ✅ Train model and measure time
        start_time = time.time()
        trainer.train()
        training_time = time.time() - start_time

        logger.info(f"POPT training completed in {training_time:.2f} seconds")

        # ✅ Save the model
        model_save_path = os.path.join(CONFIG["output_dir"], "popt_model")
        model.save_pretrained(model_save_path)
        tokenizer.save_pretrained(model_save_path)

        # ✅ Evaluate the model (limit eval to 100 samples)
        metrics, samples = evaluate_model(model, tokenizer, eval_data, "POPT", max_eval_samples=100)
        metrics["training_time"] = training_time

        # ✅ Save results
        results = {
            "method": "POPT",
            "training_metrics": {
                "loss": trainer.metrics_history.get('loss', []),
                "kl_div": trainer.metrics_history.get('kl_div', []),
                "reward": trainer.metrics_history.get('reward', [])
            },
            "evaluation_metrics": metrics,
            "samples": samples,
            "training_time": training_time,
            "model_size": sum(p.numel() for p in model.parameters()),
            "parameters": {
                "beta": CONFIG["beta"],
                "learning_rate": CONFIG["learning_rate"],
                "batch_size": 2,  # ✅ Small batch size
                "epochs": CONFIG["num_train_epochs"]
            }
        }

        with open(os.path.join(CONFIG["output_dir"], "popt_results.json"), "w") as f:
            json.dump(results, f, indent=2)

        # ✅ Print Training Time
        print("\n=== POPT Training Time ===")
        print(f"{training_time:.2f} seconds\n")

        # ✅ Print Evaluation Metrics
        print("\n=== POPT Evaluation Metrics ===")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")

        # ✅ Print Generated Samples
        print("\n=== Sample Generated Outputs (POPT) ===")
        for i, sample in enumerate(samples[:5]):  # Print only first 5 samples
            print(f"\nSample {i+1}:")
            print(f"Prompt: {sample['prompt']}")
            print(f"Reference: {sample['reference']}")
            print(f"Model Output: {sample['generation']}")

        return model, metrics, results

    except Exception as e:
        logger.error(f"Error in POPT training: {e}")
        logger.exception("Details:")

        # ✅ Create error results
        results = {
            "method": "POPT",
            "error": str(e),
            "training_time": 0.0,
            "evaluation_metrics": {
                "rouge1": 0.0,
                "rouge2": 0.0,
                "rougeL": 0.0
            }
        }

        return None, None, results

# Evaluation Metrics

For evaluation, we are computing ROUGE scores, which tell us how close the text generated by the fine-tuned models is to the text created by humans.

We also keep track of training time, generation time, and tokens per second

In [ ]:
# ===== EVALUATION =======

def compute_rouge_metrics(predictions, references, tokenizer):
    """Compute ROUGE metrics for generated text"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }

    for pred, ref in zip(predictions, references):
        results = scorer.score(ref, pred)
        scores['rouge1'].append(results['rouge1'].fmeasure)
        scores['rouge2'].append(results['rouge2'].fmeasure)
        scores['rougeL'].append(results['rougeL'].fmeasure)

    # Calculate averages
    metrics = {
        'rouge1': np.mean(scores['rouge1']),
        'rouge2': np.mean(scores['rouge2']),
        'rougeL': np.mean(scores['rougeL'])
    }

    return metrics

def evaluate_model(model, tokenizer, eval_data, method_name, max_eval_samples=100):
    """Evaluate a model on a subset of preference data (max 100 samples)"""
    logger.info(f"Evaluating {method_name} model")
    model.eval()

    try:
        # ✅ Select up to 100 evaluation samples
        num_samples = min(max_eval_samples, len(eval_data))
        eval_samples = eval_data.select(range(num_samples))  # ✅ Use .select() instead of slicing

        # ✅ Convert dataset to a list of dictionaries if necessary
        if isinstance(eval_samples, Dataset):
            eval_samples = eval_samples.to_list()

        # ✅ Debugging print to check structure
        print(f"Eval Samples Type: {type(eval_samples)}")
        print(f"First Sample: {eval_samples[0]}") if eval_samples else print("No samples available.")

        # ✅ Extract prompts correctly
        prompts = []
        references = []

        for item in eval_samples:
            try:
                # ✅ Extract the first message from "chosen" that has role "user"
                user_message = next(msg["content"] for msg in item["chosen"] if msg["role"] == "user")
                assistant_response = next(msg["content"] for msg in item["chosen"] if msg["role"] == "assistant")

                prompts.append(user_message)
                references.append(assistant_response)

            except Exception as e:
                logger.warning(f"Skipping a sample due to missing keys: {e}")
                continue  # Skip this sample if it's malformed

    except Exception as e:
        logger.error(f"Error processing evaluation dataset: {e}")
        return {}, []  # ✅ FIXED: Return empty metrics and samples instead of None

    # Set up generation pipeline
    gen_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1,
    )

    # Generate responses and measure time
    start_time = time.time()
    generations = []

    for prompt in prompts:
        try:
            # Tokenize with attention mask
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                return_attention_mask=True
            ).to(device)

            # Generate text
            output = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=True,
                temperature=0.7,
                num_return_sequences=1
            )

            # Decode generated text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Extract only the newly generated part
            if generated_text.startswith(prompt):
                generated_text = generated_text[len(prompt):].strip()

            generations.append(generated_text)

        except Exception as e:
            logger.error(f"Error generating response: {e}")
            generations.append("")

    generation_time = time.time() - start_time

    # Compute metrics
    metrics = compute_rouge_metrics(generations, references, tokenizer)
    metrics["generation_time"] = generation_time
    metrics["tokens_per_second"] = sum(len(gen.split()) for gen in generations) / generation_time

    # Log results
    logger.info(f"=== {method_name} Evaluation Results ===")
    for k, v in metrics.items():
        logger.info(f"{k}: {v:.4f}")

    # Save generated samples
    samples = []
    for i in range(min(5, len(generations))):
        samples.append({
            "prompt": prompts[i],
            "generation": generations[i],
            "reference": references[i]
        })

    return metrics, samples

# Main function

In [ ]:
# ===== MAIN FUNCTION =====
def run_benchmark():
    """Run DPO training benchmark"""
    logger.info(f"Starting preference learning benchmark at {datetime.now()}")
    logger.info(f"Using model: {CONFIG['model_name']}")
    logger.info(f"Using dataset: {CONFIG['dataset_name']}")

    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Prepare datasets
        train_data, eval_data = prepare_datasets(tokenizer)

        # Run DPO training
        logger.info("\n=== Running DPO Benchmark ===")
        model, metrics, results = train_dpo(tokenizer, train_data, eval_data)

        # ✅ Print training time
        print("\n=== Training Time ===")
        print(f"{results['training_time']:.2f} seconds\n")

        # ✅ Print Evaluation Metrics (ROUGE Scores)
        print("\n=== Evaluation Metrics ===")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")

        # Clean up GPU memory
        torch.cuda.empty_cache()
        gc.collect()

        logger.info("\n=== Running POPT Benchmark ===")
        model, metrics, results = train_popt(tokenizer, train_data, eval_data)

        # ✅ Print training time
        print("\n=== Training Time ===")
        print(f"{results['training_time']:.2f} seconds\n")

        # ✅ Print Evaluation Metrics (ROUGE Scores)
        print("\n=== Evaluation Metrics ===")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")

        # Clean up GPU memory
        torch.cuda.empty_cache()
        gc.collect()

        logger.info(f"Benchmark completed at {datetime.now()}")
        logger.info(f"Results saved to {CONFIG['output_dir']}")
        return metrics, results

    except Exception as e:
        logger.error(f"Error in benchmark: {e}")
        logger.exception("Details:")

# Run benchmark

In lines 10-22, there is a big config that the other functions use. The number of epochs could be increased, the dataset could be changed, the model name could be changed, etc.

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

# Configuration
CONFIG = {
    "model_name": "Qwen/Qwen2-0.5B-Instruct",
    "dataset_name": "trl-lib/ultrafeedback_binarized",  # ✅ Use ultrafeedback_binarized
    "dataset_split": "train",  # ✅ Use full training set, but filter later
    "num_train_samples": 500,  # ✅ Train on only 500 samples
    "eval_samples": 50,  # ✅ Evaluation set (optional)
    "output_dir": "./benchmark_results",
    "max_length": 256,
    "batch_size": 4,
    "learning_rate": 1e-4,
    "num_train_epochs": 1,
    "beta": 1.0,  # KL regularization strength
}

# Create output directory
os.makedirs(CONFIG["output_dir"], exist_ok=True)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

metrics, results = run_benchmark()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-28-155ec0a12443>:212: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  dpo_trainer = DPOTrainer(


Extracting prompt in train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Step,Training Loss
10,10.819300
20,50.502800
30,50.072800
40,70.459900
50,65.751200
60,40.054000


Device set to use cuda:0


Eval Samples Type: <class 'list'>
First Sample: {'chosen': [{'content': 'Create a comprehensive social media plan that outlines the objectives, target audience, platforms, messaging, content strategy, tone of voice, frequency of posts, and metrics for measuring success for a new initiative. The plan should also include a timeline for implementation and a budget allocation for paid social media advertising.', 'role': 'user'}, {'content': 'Title: Comprehensive Social Media Plan for a New Initiative\n\nI. Objectives:\n   A. Increase brand awareness and visibility\n   B. Generate a loyal and engaged community\n   C. Drive traffic to the website\n   D. Generate leads and sales\n   E. Establish ourselves as thought leaders in the industry\n\nII. Target Audience:\n   A. Demographics\n      1. Age group: 24-45\n      2. Gender: Male and female\n      3. Location: United States\n      4. Occupation: Professionals and entrepreneurs\n   B. Psychographics\n      1. Interests: Technology, innovatio

In [ ]:
print(metrics, results)

{'rouge1': 0.14401077881391675, 'rouge2': 0.04014290703139522, 'rougeL': 0.09967850593229491, 'generation_time': 91.14116072654724, 'tokens_per_second': 21.373438570138752, 'training_time': 66.77635622024536} {'method': 'DPO', 'evaluation_metrics': {'rouge1': 0.14401077881391675, 'rouge2': 0.04014290703139522, 'rougeL': 0.09967850593229491, 'generation_time': 91.14116072654724, 'tokens_per_second': 21.373438570138752, 'training_time': 66.77635622024536}, 'samples': [{'prompt': 'Create a comprehensive social media plan that outlines the objectives, target audience, platforms, messaging, content strategy, tone of voice, frequency of posts, and metrics for measuring success for a new initiative. The plan should also include a timeline for implementation and a budget allocation for paid social media advertising.', 'generation': "To create a comprehensive social media platform, you would need to focus on your target audience and understand how they are connected with your brand. Here's a sa

In [ ]:
!pip install -U trl

  Using cached trl-0.15.2-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
Using cached trl-0.15.2-py3-none-any.whl (318 kB)
Using cached datasets-3.3.2-py3-none-any.whl (485 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.5
    Uninstalling datasets-2.14.5:
      Successfully uninstalled datasets-2.14.5
  Attempting uninstall: trl
    Found existing installation: trl 0.7.2
    Uninstalling trl-0.7.2:
      Successfully uninstalled trl-0.7.2
